In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

'''
split into testing and training data

PTS_TOTAL_OVER_SB is the outcome column (1=over, 0=under)
'''

csv_input = input('Which set should be used in predicting?\n 1.Normal data \n 2.Replace missing scorebook values with random \n 3.Replace all scorebook values with fixed number?');

if csv_input == '1':
    csv_string = 'prediction_data_unreplaced.csv';
elif csv_input == '2':
    csv_string = 'prediction_data_random_sb.csv';
elif csv_input == '3':
    csv_string = 'prediction_data_fixed_sb.csv';
else:
    csv_string = '';
    
df = pd.read_csv('../output_files/' + csv_string).dropna(axis=1).dropna(axis=0)

remove_cols = ['PTS_TOTAL_OVER_SB', 'PTS_TOTAL'];
X = df.drop(columns=remove_cols)

y = df['PTS_TOTAL_OVER_SB']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

Which set should be used in predicting?
 1.Normal data 
 2.Replace missing scorebook values with random 
 3.Replace all scorebook values with fixed number?1


## Random Forest

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestClassifier(n_estimators = 1600, min_samples_split = 10, min_samples_leaf = 1, max_features = 'sqrt', max_depth = None, bootstrap = True)

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(X_train, y_train)

rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\addya\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
123 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
89 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\addya\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\addya\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Users\addya\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\addya\anaconda3\Lib\site-package

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 110,
 'bootstrap': True}

In [ ]:
import numpy as np
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
)
from sklearn.model_selection import KFold, cross_val_score

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 1600, min_samples_split = 10, min_samples_leaf = 1, max_features = 'sqrt', max_depth = None, bootstrap = True)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, rf_pred)
print("Accuracy:", accuracy)

clf_rep_rf = classification_report(y_test, rf_pred, output_dict=True)
pd.DataFrame(clf_rep_rf).T

In [9]:
from sklearn.model_selection import KFold, cross_val_score

rf_clf = RandomForestClassifier(n_estimators = 1600, min_samples_split = 10, min_samples_leaf = 1, max_features = 'sqrt', max_depth = None, bootstrap = True)

k_folds = KFold(n_splits = 10)

cvscores_rf = cross_val_score(rf_clf, X, y, cv = k_folds) 

print("Cross Validation Scores: ", cvscores_rf)
print("Average CV Score: ", cvscores_rf.mean())
print("Number of CV Scores used in Average: ", len(cvscores_rf)) 

print("%0.2f accuracy with a standard deviation of %0.2f" % (cvscores_rf.mean(), cvscores_rf.std()))

Cross Validation Scores:  [0.52360515 0.527897   0.48927039 0.472103   0.5        0.5387931
 0.52155172 0.51293103 0.5387931  0.54741379]
Average CV Score:  0.5172358295101376
Number of CV Scores used in Average:  10
0.52 accuracy with a standard deviation of 0.02
